In [15]:
##### import the necessary modules and set chart style####
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
mpl.style.use('bmh')
import pandas_datareader.data as web
import matplotlib.pylab as plt
from datetime import datetime
import statsmodels.api as sm
from pykalman import KalmanFilter
from math import sqrt

In [20]:
stock_list = pd.read_html("https://finance.yahoo.com/screener/predefined/ms_technology?offset=0&count=100")[0]
print(stock_list)

   Symbol                                               Name  \
0    AAPL                                         Apple Inc.   
1    MSFT                              Microsoft Corporation   
2    GOOG                                      Alphabet Inc.   
3   GOOGL                                      Alphabet Inc.   
4      FB                                     Facebook, Inc.   
5     TSM  Taiwan Semiconductor Manufacturing Company Lim...   
6    INTC                                  Intel Corporation   
7    CSCO                                Cisco Systems, Inc.   
8    ORCL                                 Oracle Corporation   
9     SAP                                             SAP SE   
10   ADBE                                         Adobe Inc.   
11    CRM                               salesforce.com, inc.   
12    TXN                     Texas Instruments Incorporated   
13   NVDA                                 NVIDIA Corporation   
14    IBM        International Business 

In [51]:
stocks = pd.DataFrame(stock_list)
stocks["Symbol"]
df_list = stocks["Symbol"]
df = df_list.astype(str)
df[:4]

0     AAPL
1     MSFT
2     GOOG
3    GOOGL
Name: Symbol, dtype: object

In [52]:
#scrape html from website and store 3rd DataFrame as our stock tickers - this is dictated to us by the structure of the html
#stock_list = pd.read_html("https://www.marketwatch.com/tools/industry/stocklist.asp?bcind_ind=9535&bcind_period=3mo")[0]
#convert the DataFrame of stocks into a list so we can easily iterate over it
#stocks = stock_list[1].dropna()[1:].tolist()
#set empty list o hold the stock price DataFrames that we can later concatenate into a master frame
#df_list = []
#not all stocks will return data so set up an empty list to store the stock tickers that actually successfully returns data
#used_stocks = []
#iterate over stock tickers in list and download relevant data, storing said data and successfully downloaded tickers along the way
#for stock in stocks:
  #  try:
  #      data = pd.DataFrame(web.DataReader(stock,data_source='iex',start='01/01/2017')['close'])
  #      data.columns = [stock]
  #      df_list.append(data)
  #      used_stocks.append(stock)
  #  except:
  #      pass
#concatenate list of individual tciker price DataFrames into one master DataFrame
#df = pd.concat(df_list,axis=1)

In [55]:
from backtester.dataSource.yahoo_data_source import YahooStockDataSource
from datetime import datetime

startDateStr = '2018/01/01'
endDateStr = '2019/10/21'
cachedFolderName = 'yahooData/'
dataSetId = 'testPairsTrading'
instrumentIds = df
ds = YahooStockDataSource(cachedFolderName=cachedFolderName,
                            dataSetId=dataSetId,
                            instrumentIds=instrumentIds,
                            startDateStr=startDateStr,
                            endDateStr=endDateStr,
                            event='history')
data = ds.getBookDataByFeature()['adjClose']

Processing data for stock: AAPL
Processing data for stock: MSFT
Processing data for stock: GOOG
Processing data for stock: GOOGL
Processing data for stock: FB
Processing data for stock: TSM
Processing data for stock: INTC
Processing data for stock: CSCO
Processing data for stock: ORCL
Processing data for stock: SAP
Processing data for stock: ADBE
Processing data for stock: CRM
Processing data for stock: TXN
Processing data for stock: NVDA
Processing data for stock: IBM
Processing data for stock: ACN
Processing data for stock: AVGO
Processing data for stock: ASML
Processing data for stock: QCOM
Processing data for stock: SNE
Processing data for stock: INTU
Processing data for stock: VMW
Processing data for stock: UBER
Processing data for stock: MU
Processing data for stock: AMAT
Processing data for stock: LHX
Processing data for stock: NOW
Processing data for stock: ATVI
Processing data for stock: ADI
Processing data for stock: INFY
Processing data for stock: DELL
Processing data for st

In [61]:
# Heatmap to show the p-values of the cointegration test
# between each pair of stocks
data_clean = data.dropna()
scores, pvalues, pairs = find_cointegrated_pairs(data_clean)
import seaborn
#m = [0,0.2,0.4,0.6,0.8,1]
#m = np.linspace(0,1,6)
m = np.linspace()
seaborn.heatmap(pvalues, xticklabels=instrumentIds, 
                yticklabels=instrumentIds, cmap='RdYlGn_r' 
                , mask = (pvalues >= 0.05)
                )
plt.show()
print("Co-integrated pairs found:   ", pairs)

C:\Users\samar\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1636: RuntimeWarning:

invalid value encountered in double_scalars

C:\Users\samar\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1432: CollinearityWarning:

y0 and y1 are (almost) perfectly colinear.Cointegration test is not reliable in this case.

C:\Users\samar\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1636: RuntimeWarning:

divide by zero encountered in double_scalars

C:\Users\samar\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:867: RuntimeWarning:

divide by zero encountered in log

C:\Users\samar\Anaconda3\lib\site-packages\statsmodels\base\model.py:1294: RuntimeWarning:

invalid value encountered in true_divide

C:\Users\samar\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning:

invalid value encountered in greater

C:\Users\samar\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWa

ValueError: not enough values to unpack (expected 3, got 2)

In [63]:
#NOTE CRITICAL LEVEL HAS BEEN SET TO 5% FOR COINTEGRATION TEST
def find_cointegrated_pairs(dataframe, critial_level = 0.05):
    n = dataframe.shape[1] # the length of dateframe
    pvalue_matrix = np.ones((n, n)) # initialize the matrix of p
    keys = dataframe.columns # get the column names
    pairs = [] # initilize the list for cointegration
    for i in range(n):
        for j in range(i+1, n): # for j bigger than i
            stock1 = dataframe[keys[i]] # obtain the price of "stock1"
            stock2 = dataframe[keys[j]]# obtain the price of "stock2"
            result = sm.tsa.stattools.coint(stock1, stock2) # get conintegration
            pvalue = result[1] # get the pvalue
            pvalue_matrix[i, j] = pvalue
            if pvalue < critial_level: # if p-value less than the critical level
                pairs.append((keys[i], keys[j], pvalue)) # record the contract with that p-value
    return pvalue_matrix, pairs
pairs = find_cointegrated_pairs(data_clean, critial_level= 0.05)


C:\Users\samar\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1636: RuntimeWarning:

invalid value encountered in double_scalars

C:\Users\samar\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1432: CollinearityWarning:

y0 and y1 are (almost) perfectly colinear.Cointegration test is not reliable in this case.

C:\Users\samar\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1636: RuntimeWarning:

divide by zero encountered in double_scalars

C:\Users\samar\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:867: RuntimeWarning:

divide by zero encountered in log

C:\Users\samar\Anaconda3\lib\site-packages\statsmodels\base\model.py:1294: RuntimeWarning:

invalid value encountered in true_divide

C:\Users\samar\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning:

invalid value encountered in greater

C:\Users\samar\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWa

(array([[1.        , 0.        , 1.        , ..., 0.94867693, 0.23760606,
         0.99395504],
        [1.        , 1.        , 0.08479233, ..., 0.9916792 ,        nan,
                nan],
        [1.        , 1.        , 1.        , ..., 0.24248368, 0.40192293,
         0.20042186],
        ...,
        [1.        , 1.        , 1.        , ..., 1.        , 0.9899033 ,
         0.03206383],
        [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
         0.86267732],
        [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
         1.        ]]),
 [('AAPL', 'MSFT', 0.0),
  ('AAPL', 'CRM', 0.0),
  ('AAPL', 'QCOM', 0.0),
  ('AAPL', 'NTES', 0.0),
  ('MSFT', 'XLNX', 4.5442868094471976e-14),
  ('MSFT', 'SSNC', 0.0),
  ('GOOG', 'SAP', 0.00035011037837011117),
  ('GOOG', 'LHX', 0.009053380875921528),
  ('GOOG', 'TWTR', 0.011185817191985447),
  ('GOOG', 'SNAP', 6.560295752055094e-05),
  ('GOOG', 'CHKP', 0.04903498023191316),
  ('GOOG', 'AKAM', 0.0082454228

In [ ]:
#set up the split point for our "training data" on which to perform the co-integration test (the remaining dat awill be fed to our backtest function)
split = int(len(df) * .4)
#run our dataframe (up to the split point) of ticker price data through our co-integration function and store results
pvalue_matrix,pairs = find_cointegrated_pairs(df[:split])
#convert our matrix of stored results into a DataFrame
pvalue_matrix_df = pd.DataFrame(pvalue_matrix)
#use Seaborn to plot a heatmap of our results matrix
fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(pvalue_matrix_df,xticklabels=used_stocks,yticklabels=used_stocks,ax=ax)

In [12]:
for pair in pairs:
    print("Stock {} and stock {} has a co-integration score of {}".format(pair[0],pair[1],round(pair[2],4)))

NameError: name 'pairs' is not defined

In [13]:
def KalmanFilterAverage(x):
  # Construct a Kalman filter
    kf = KalmanFilter(transition_matrices = [1],
    observation_matrices = [1],
    initial_state_mean = 0,
    initial_state_covariance = 1,
    observation_covariance=1,
    transition_covariance=.01)
  # Use the observed values of the price to get a rolling mean
    state_means, _ = kf.filter(x.values)
    state_means = pd.Series(state_means.flatten(), index=x.index)
    return state_means
# Kalman filter regression
def KalmanFilterRegression(x,y):
    delta = 1e-3
    trans_cov = delta / (1 - delta) * np.eye(2) # How much random walk wiggles
    obs_mat = np.expand_dims(np.vstack([[x], [np.ones(len(x))]]).T, axis=1)
    kf = KalmanFilter(n_dim_obs=1, n_dim_state=2, # y is 1-dimensional, (alpha, beta) is 2-dimensional
    initial_state_mean=[0,0],
    initial_state_covariance=np.ones((2, 2)),
    transition_matrices=np.eye(2),
    observation_matrices=obs_mat,
    observation_covariance=2,
    transition_covariance=trans_cov)
    # Use the observations y to get running estimates and errors for the state parameters
    state_means, state_covs = kf.filter(y.values)
    return state_means
def half_life(spread):
    spread_lag = spread.shift(1)
    spread_lag.iloc[0] = spread_lag.iloc[1]
    spread_ret = spread - spread_lag
    spread_ret.iloc[0] = spread_ret.iloc[1]
    spread_lag2 = sm.add_constant(spread_lag)
    model = sm.OLS(spread_ret,spread_lag2)
    res = model.fit()
    halflife = int(round(-np.log(2) / res.params[1],0))
    if halflife <= 0:
        halflife = 1
    return halflife

In [14]:
def backtest(df,s1, s2):
    #############################################################
    # INPUT:
    # DataFrame of prices
    # s1: the symbol of contract one
    # s2: the symbol of contract two
    # x: the price series of contract one
    # y: the price series of contract two
    # OUTPUT:
    # df1['cum rets']: cumulative returns in pandas data frame
    # sharpe: Sharpe ratio
    # CAGR: Compound Annual Growth Rate
    x = df[s1]
    y = df[s2]
    # run regression (including Kalman Filter) to find hedge ratio and then create spread series
    df1 = pd.DataFrame({'y':y,'x':x})
    df1.index = pd.to_datetime(df1.index)
    state_means = KalmanFilterRegression(KalmanFilterAverage(x),KalmanFilterAverage(y))
    df1['hr'] = - state_means[:,0]
    df1['spread'] = df1.y + (df1.x * df1.hr)
    # calculate half life
    halflife = half_life(df1['spread'])
    # calculate z-score with window = half life period
    meanSpread = df1.spread.rolling(window=halflife).mean()
    stdSpread = df1.spread.rolling(window=halflife).std()
    df1['zScore'] = (df1.spread-meanSpread)/stdSpread
    ##############################################################
    # trading logic
    entryZscore = 2
    exitZscore = 0
    #set up num units long
    df1['long entry'] = ((df1.zScore < - entryZscore) & ( df1.zScore.shift(1) > - entryZscore))
    df1['long exit'] = ((df1.zScore > - exitZscore) & (df1.zScore.shift(1) < - exitZscore)) df1['num units long'] = np.nan df1.loc[df1['long entry'],'num units long'] = 1 df1.loc[df1['long exit'],'num units long'] = 0 df1['num units long'][0] = 0 df1['num units long'] = df1['num units long'].fillna(method='pad') #set up num units short df1['short entry'] = ((df1.zScore > entryZscore) & ( df1.zScore.shift(1) < entryZscore))
    df1['short exit'] = ((df1.zScore < exitZscore) & (df1.zScore.shift(1) > exitZscore))
    df1.loc[df1['short entry'],'num units short'] = -1
    df1.loc[df1['short exit'],'num units short'] = 0
    df1['num units short'][0] = 0
    df1['num units short'] = df1['num units short'].fillna(method='pad')
    df1['numUnits'] = df1['num units long'] + df1['num units short']
    df1['spread pct ch'] = (df1['spread'] - df1['spread'].shift(1)) / ((df1['x'] * abs(df1['hr'])) + df1['y'])
    df1['port rets'] = df1['spread pct ch'] * df1['numUnits'].shift(1)
    df1['cum rets'] = df1['port rets'].cumsum()
    df1['cum rets'] = df1['cum rets'] + 1
    ##############################################################
    try:
        sharpe = ((df1['port rets'].mean() / df1['port rets'].std()) * sqrt(252))
    except ZeroDivisionError:
        sharpe = 0.0
    ##############################################################
    start_val = 1
    end_val = df1['cum rets'].iat[-1]
    start_date = df1.iloc[0].name
    end_date = df1.iloc[-1].name
    days = (end_date - start_date).days
    CAGR = round(((float(end_val) / float(start_val)) ** (252.0/days)) - 1,4)
    df1[s1+ " "+s2] = df1['cum rets']
    return df1[s1+" "+s2], sharpe, CAGR

SyntaxError: invalid syntax (<ipython-input-14-8a074bcd2459>, line 33)

In [15]:
results = []
for pair in pairs:
    rets, sharpe,  CAGR = backtest(df[split:],pair[0],pair[1])
    results.append(rets)
    print("The pair {} and {} produced a Sharpe Ratio of {} and a CAGR of {}".format(pair[0],pair[1],round(sharpe,2),round(CAGR,4)))
    rets.plot(figsize=(20,15),legend=True)

NameError: name 'pairs' is not defined

In [16]:
#concatenate together the individual equity curves into a single DataFrame
results_df = pd.concat(results,axis=1).dropna()
#equally weight each equity curve by dividing each by the number of pairs held in the DataFrame
results_df /= len(results_df.columns)
#sum up the equally weighted equity curves to get our final equity curve
final_res = results_df.sum(axis=1)
#plot the chart of our final equity curve
final_res.plot(figsize=(20,15))

ValueError: No objects to concatenate

In [18]:
#calculate and print our some final stats for our combined equity curve
sharpe = (final_res.pct_change().mean() / final_res.pct_change().std()) * (sqrt(252))
start_val = 1
end_val = final_res.iloc[-1]
start_date = final_res.index[0]
end_date = final_res.index[-1]
days = (end_date - start_date).days
CAGR = round(((float(end_val) / float(start_val)) ** (252.0/days)) - 1,4)
print("Sharpe Ratio is {} and CAGR is {}".format(round(sharpe,2),round(CAGR,4)))

NameError: name 'final_res' is not defined